In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_path ='../input/tabular-playground-series-feb-2022/train.csv'
test_path = '../input/tabular-playground-series-feb-2022/test.csv'
sub_path = '../input/tabular-playground-series-feb-2022/sample_submission.csv'

In [ ]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_sub = pd.read_csv(sub_path)

In [ ]:
df_sub.sample(6)

In [ ]:
df_train

In [ ]:
df_train['target'].value_counts().index

In [ ]:
df_train.info()

In [ ]:
T = df_train[[x for x in list(df_train.columns) if x != 'target']]
df_unified = pd.concat([T , df_test],axis = 0 ,ignore_index = True )
df_unified

In [ ]:
x = []
for column in list(df_unified.columns):
    if df_unified[column].isnull().sum()!=0:
        x.append(column)
len(x)

1. Here no columns have nan value

In [ ]:
df_unified.drop('row_id',axis = 1, inplace = True)

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(df_train.target)


1. The target column is not imbalanced

In [ ]:
from sklearn.preprocessing import RobustScaler

for column in list(df_unified.columns):
    rs = RobustScaler()
    df_unified[column] = rs.fit_transform(df_unified[column].values.reshape(-1,1))

In [ ]:
from sklearn.preprocessing import QuantileTransformer

for column in list(df_unified.columns):
    qt = QuantileTransformer(output_distribution='normal')
    df_unified[column] = qt.fit_transform(df_unified[column].values.reshape(-1,1))


In [ ]:
df_train['target'].value_counts()

In [ ]:
target_encode = {'Bacteroides_fragilis':1,'Streptococcus_pyogenes':2,
                'Streptococcus_pneumoniae':3, 'Campylobacter_jejuni':4,
                'Salmonella_enterica':5 , 'Escherichia_coli':6,
                'Enterococcus_hirae':7 , 'Escherichia_fergusonii':8,
                'Staphylococcus_aureus':9 , 'Klebsiella_pneumoniae':10}
df_train['target'] = df_train['target'].map(target_encode)
df_train['target'].value_counts()

In [ ]:
training = df_unified.head(df_train.shape[0])
testing = df_unified.tail(df_test.shape[0])

X = training
y = df_train['target']

In [ ]:
from sklearn.model_selection import train_test_split

X_train , x_valid , Y_train , y_valid = train_test_split(X,y,test_size = 0.2 ,
                                                        stratify = y ,random_state = 42 )

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
extr_clf = BaggingClassifier(base_estimator = ExtraTreesClassifier(), n_estimators = 10)
extr_clf.fit(X_train , Y_train)


In [ ]:
# from xgboost import XGBClassifier
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.metrics import roc_auc_score

# xgbc_clf = XGBClassifier(eval_metric=['auc'])
# xgbc_clf.fit(X_train , Y_train , eval_set=[[X_train,Y_train],[x_valid,y_valid]])

# y_valid_preds = xgbc_clf.predict(x_valid)



In [ ]:
# from catboost import CatBoostClassifier
# from sklearn.metrics import roc_auc_score

# cbc = CatBoostClassifier(n_estimators=125, auto_class_weights= 'Balanced',eval_metric='AUC')

# cbc.fit(X_train,Y_train,eval_set=(x_valid,y_valid))



In [ ]:
# T_cbc = cbc.predict(testing)

In [ ]:
T_extr= extr_clf.predict(testing)

In [ ]:
# T = xgbc_clf.predict(testing)

In [ ]:
df_test['target'] = -1
df_test['target'] = T_extr
submission = df_test[['row_id','target']]
submission

In [ ]:
reverse_encode = {1:'Bacteroides_fragilis',2:'Streptococcus_pyogenes',
                3:'Streptococcus_pneumoniae', 4:'Campylobacter_jejuni',
                5:'Salmonella_enterica' , 6:'Escherichia_coli',
                7:'Enterococcus_hirae' , 8:'Escherichia_fergusonii',
                9:'Staphylococcus_aureus' , 10:'Klebsiella_pneumoniae'}
submission['target'] = submission['target'].map(reverse_encode)

In [ ]:
submission

In [ ]:
submission.to_csv('bagging_extra.csv',index =False)

In [ ]:
# submission.to_csv('baseline_cbc.csv',index = False)